# The value of the multi-exposure visit

## author:
- **David W. Hogg** (Flatiron)

## projects:
- Figure out how splitting an exposure into sub-exposures helps or hurts.
- Look at information gain per unit time on the mean function and (parameters of) the variance function.
- Use the magic of `jax` to take derivatives of, or optimize, strategies.

## notes:
- Using SI units, of course; not using strict units checking, but maybe should be? For example, I don't have a good explanation for the amplitude that I am using for the p-modes amplitude.
- Equations have not been carefully checked. There could be factors of 2 or pi, and maybe also just bugs.
- Check that derivatives can be taken and visualized.

In [ ]:
import jax
jax.config.update("jax_enable_x64", True)
from jax import vmap
import jax.numpy as jnp
import pylab as plt

In [ ]:
# set up the assumptions of the method
READ_TIME = 30. # seconds; optimistic
SLEW_TIME = 120. # seconds; very optimistic
PMODE_AMPLITUDE = 1.0e4 # m^2 s^{-1} (variance times time, or variance per unit frequency)
PMODE_NUMAX = 1. / (5.2 * 60.) # Hz; making this up
PMODE_QEFF = 8. # making this up
WHITE_NOISE_AMPLITUDE = 200. # m^2 s^{-1} (variance times time)
READ_NOISE_TIME = 1. # seconds; integration time at which white noise equals read noise
SATURATION_TIME = 1200. # s; exposure time at which saturation starts to matter

In [ ]:
# make a frequency grid on which all integrals will be done
day = 86400 # seconds
DF = 1. / (5. * day) # in Hz; fine enough to resolve the individual modes maybe
fmax = 0.01 # Hz
FS = jnp.arange(DF, fmax, DF)
print(len(FS))

In [ ]:
def pmodes(fs, A = PMODE_AMPLITUDE):
    """
    - `A` has units of RV variance times time.
    """
    numax = PMODE_NUMAX
    nuw = numax / PMODE_QEFF
    return A * jnp.exp(-0.5 * (fs - numax) ** 2 / nuw ** 2)

In [ ]:
# BUG: I just typed these from memory and they are probably wrong!

def one_exposure_variance(Texp):
    power = pmodes(FS)
    return 2. * jnp.sum(DF * power
                     * (jnp.sin(jnp.pi * FS * Texp) / (jnp.pi * FS * Texp)) ** 2) \
           + WHITE_NOISE_AMPLITUDE / Texp \
           + WHITE_NOISE_AMPLITUDE * READ_NOISE_TIME / Texp ** 2

def two_exposure_covariance(Texp1, Texp2, DeltaT):
    """
    ## notes:
    - Assumes (requires) `DeltaT > Texp1`.
    
    ## bugs:
    - Because this is `vmap`ed below, it can't `assert` if `DeltaT < Texp1`.
    """
    return 2. * jnp.sum(DF * pmodes(FS)
                     * (jnp.sin(jnp.pi * FS * Texp1) / (jnp.pi * FS * Texp1))
                     * (jnp.sin(jnp.pi * FS * Texp2) / (jnp.pi * FS * Texp2))
                     * jnp.cos(2. * jnp.pi * FS * DeltaT))

In [ ]:
def exptime(T, N):
    T1 = T - max(READ_TIME, SLEW_TIME)
    return (T1 + READ_TIME) / N - READ_TIME

def one_visit_variance(T, N):
    Texp = exptime(T, N)
    DeltaT = Texp + READ_TIME
    covar = jnp.diag(jnp.zeros(N) + one_exposure_variance(Texp))
    for dd in range(1,N):
        DeltaT = dd * (Texp + READ_TIME)
        covarij = two_exposure_covariance(Texp, Texp, DeltaT)
        for i in range(N):
            j = i + dd
            if j < N:
                covar = covar.at[i, j].set(covarij)
                covar = covar.at[j, i].set(covarij)
    return covar

In [ ]:
def one_visit_information(T, N):
    return jnp.dot(jnp.ones(N), jnp.linalg.solve(one_visit_variance(T, N), jnp.ones(N)))

In [ ]:
Ns = [1, 2, 3, 4, 5, 7, 10]
times = jnp.arange(200., 7200., 10.)
allinfos = ()
for N in Ns:
    vectorized_visit_information = vmap(one_visit_information, (0, None), 0)
    infos = vectorized_visit_information(times, N)
    vectorized_exptime = vmap(exptime, (0, None), 0)
    exptimes = vectorized_exptime(times, N)
    infos = infos.at[jnp.where(exptimes < 1.)].set(jnp.nan) # don't trust < 20 sec exposures
    infos = infos.at[jnp.where(exptimes > SATURATION_TIME)].set(jnp.nan) # don't trust < 20 sec exposures
    allinfos += (infos, )

In [ ]:
for i, N in enumerate(Ns):
    sigmas = 1. / allinfos[i]
    plt.plot(times - max(SLEW_TIME, READ_TIME), sigmas, label=N)
plt.loglog()
plt.legend()
plt.xlabel("total time (excluding final read or slew) (s)")
plt.ylabel("expected uncertainty on the mean (m/s)")
plt.savefig("sigma_v_time.png")

In [ ]:
maxrate = 0.
for i, N in enumerate(Ns):
    info_rates = allinfos[i] / times
    if jnp.nanmax(info_rates) > maxrate:
        maxrate = jnp.nanmax(info_rates)
    plt.plot(times, info_rates, label=N)
plt.legend()
plt.loglog()
plt.xlabel("total wall-clock time (including final read or slew) (s)")
plt.ylabel("information per unit wall-clock time (s / m^2)")
print(0.4 * maxrate, 1.2 * maxrate)
plt.ylim(0.4 * maxrate, 1.2 * maxrate)
plt.savefig("information_rate_v_time.png")